# Constructing a consistent US County Per Capita Income Dataset 1969-2017

This notebook builds a cleaned geopackage for use in the inequality chapter of the book.  It does so by carrying out a number of data processing steps to:

- Generalize the boundaries of the county shapefile to faciltate processing
- Joining income attributes with geometries
- Handling the birth and deaths of counties over the time series

In [ ]:
import pandas
from zipfile import ZipFile
import glob
from urllib.request import urlopen
from io import BytesIO

In [ ]:
!pip install topojson

In [ ]:
import fiona.io
import geopandas as gpd
import pandas
import requests
import topojson as tp

In [ ]:
url = 'https://github.com/gdsbook/data_archive/raw/master/us_county_income/tl_2019_us_county.zip'

response = requests.get(url)
data_bytes = response.content

In [ ]:
with fiona.io.ZipMemoryFile(data_bytes) as zip_memory_file:
    with zip_memory_file.open('tl_2019_us_county.shp') as collection:
        gdf = gpd.GeoDataFrame.from_features(collection, crs=collection.crs)
#gdf.to_file('tl_2019_us_county.shp')

In [ ]:
gdf.shape

In [ ]:
topo = tp.Topology(gdf, prequantize=False)


In [ ]:
gdf_simplified = topo.toposimplify(5).to_gdf()


In [ ]:
gdf_simplified.shape

In [ ]:
gdf_simplified.to_file('tl_2019_us_county.shp')

## Read the remote file with county income data

In [ ]:
caincurl = 'https://github.com/gdsbook/data_archive/raw/master/us_county_income/CAINC1.zip'
with urlopen(caincurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zf:
        zf.extractall()

In [ ]:
csv_files = glob.glob("*.csv")

In [ ]:
csv_files

In [ ]:
data = pandas.read_csv('CAINC1__ALL_STATES_1969_2017.csv', encoding='latin-1', 
                      skipfooter=3, engine='python')

In [ ]:
data.head()

In [ ]:
virginia = data[data.GeoFIPS.str.contains('\"51')] # select virginia
virginia.head()


In [ ]:
virginia.shape

In [ ]:
virginia.to_csv('virginia.csv')

In [ ]:
data.head(20)

In [ ]:
data.tail(10)

In [ ]:
# fips codes source: https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696
fips = """Alabama 	AL 	01
Alaska 	AK 	02
Arizona 	AZ 	04
Arkansas 	AR 	05
California 	CA 	06
Colorado 	CO 	08
Connecticut 	CT 	09
Delaware 	DE 	10
Florida 	FL 	12
Georgia 	GA 	13
Hawaii 	HI 	15
Idaho 	ID 	16
Illinois 	IL 	17
Indiana 	IN 	18
Iowa 	IA 	19
Kansas 	KS 	20
Kentucky 	KY 	21
Louisiana 	LA 	22
Maine 	ME 	23
Maryland 	MD 	24
Massachusetts 	MA 	25
Michigan 	MI 	26
Minnesota 	MN 	27
Mississippi 	MS 	28
Missouri 	MO 	29
Montana 	MT 	30
Nebraska 	NE 	31
Nevada 	NV 	32
New Hampshire 	NH 	33
New Jersey 	NJ 	34
New Mexico 	NM 	35
New York 	NY 	36
North Carolina 	NC 	37
North Dakota 	ND 	38
Ohio 	OH 	39
Oklahoma 	OK 	40
Oregon 	OR 	41
Pennsylvania 	PA 	42
Rhode Island 	RI 	44
South Carolina 	SC 	45
South Dakota 	SD 	46
Tennessee 	TN 	47
Texas 	TX 	48
Utah 	UT 	49
Vermont 	VT 	50
Virginia 	VA 	51
Washington 	WA 	53
West Virginia 	WV 	54
Wisconsin 	WI 	55
Wyoming 	WY 	56
American Samoa 	AS 	60
Guam 	GU 	66
Northern Mariana Islands 	MP 	69
Puerto Rico 	PR 	72
Virgin Islands 	VI 	78"""

In [ ]:
fips_df = pandas.DataFrame([fip.split("\t") for fip in fips.split("\n")])
fips_df.columns=["State", "Abbreviation", 'FIPS']

In [ ]:
fips_df

In [ ]:
omit_fips = [" \""+str(fip) for fip in ['02', 15, 60, 66, 69, 72, 78, 90, 91, 92, 93, 94, 95, 96, 97, 98]]
omit_fips

In [ ]:
small = data.copy()
for omit in omit_fips:
    small = small[~small.GeoFIPS.str.contains(omit)]
    

In [ ]:
small.shape

In [ ]:
data.shape

In [ ]:
geofips = set(code[:4] for code in data.GeoFIPS.values)

In [ ]:
geofips

In [ ]:
geofips = set(code[:4] for code in small.GeoFIPS.values)

In [ ]:
geofips

In [ ]:
small.shape

In [ ]:
for fip in geofips:
    sdf = small[small.GeoFIPS.str.contains(fip)]
    sdf.to_csv(fip[2:]+".csv")

We are done as all the csv files have been extracted.

In [ ]:
## old 015

In [ ]:
import pandas
import geopandas
%matplotlib inline
from zipfile import ZipFile

import glob

## Read remote shapefile with county geometries

In [ ]:
gdf = geopandas.read_file("tl_2019_us_county.shp")

## State Specific Data Frames

In [ ]:
#gdf = geopandas.read_file("zip:tl_2019_us_county.zip!tl_2019_us_county.shp")
csv_files = glob.glob("??.csv")
csv_files.sort()
csv_files.pop(0) # kick out US

In [ ]:
mismatch = []
gdfs = []
#for csv_file in csv_files[:1]:
for csv_file in csv_files:

    #print(csv_file)
    csv = pandas.read_csv(csv_file)
    st = csv_file[:2]
    st_gdf = gdf[gdf.STATEFP==st]
    csv = csv.iloc[3:] # kick out the state level records
    nc, kc = csv.shape
    if nc/3 != st_gdf.shape[0]:
        mismatch.append(st)
        print(st)
    else:
        csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]
        csv['GEOID'] = csv.GeoFIPS.astype(str)
        merged_gdf = st_gdf.merge(csv, on='GEOID')
        gdfs.append(merged_gdf)
        

In [ ]:
csv.GEOID

In [ ]:
len(gdfs)

In [ ]:
gdf = pandas.concat(gdfs)

In [ ]:
gdf.plot()

In [ ]:
gdf.shape[0]/3

In [ ]:
gdf.head()

In [ ]:
gdf.columns

In [ ]:
years = [str(year) for year in range(1969,2018)]
dtypes = list(zip(years,[gdf[year].dtype for year in years]))


In [ ]:
dtypes

## Issues

1. Virginia
2. Wisconsin
3. Object data types for 1969-2001

In [ ]:
gdf.to_file('pcincome0.shp')

In [ ]:
# old 020

## Virginia independent cities

Reference https://en.wikipedia.org/wiki/Independent_city_(United_States)

> In the United States, an independent city is a city that is not in the territory of any county or counties with exceptions noted below. Of the 41 independent U.S. cities,[1] 38 are in Virginia, whose state constitution makes them a special case. The three independent cities outside Virginia are Baltimore, Maryland; St. Louis, Missouri; and Carson City, Nevada. The U.S. Census Bureau uses counties as its base unit for presentation of statistical information, and treats independent cities as county equivalents for those purposes. The most populous of them is Baltimore, Maryland. 


### From BEA Income Data:
> Virginia combination areas consist of one or two independent cities with 1980 populations of less than 100,000 combined with an adjacent county. The county name appears first, followed by the city name(s). Separate estimates for the jurisdictions making up the combination area are not available. Bedford County, VA includes the independent city of Bedford for all years.

### Virginia
> The Commonwealth of Virginia is divided into 95 counties, along with 38 independent cities that are considered county-equivalents for census purposes. The map in this article, taken from the official United States Census Bureau site, includes Clifton Forge and Bedford as independent cities. This reflected the political reality at the time of the 2000 Census. However, both have since chosen to revert to town status. In Virginia, cities are co-equal levels of government to counties, but towns are part of counties. For some counties, for statistical purposes, the Bureau of Economic Analysis combines any independent cities with the county that it was once part of (before the legislation creating independent cities took place in 1871).

[Source](https://en.wikipedia.org/wiki/List_of_cities_and_counties_in_Virginia)

### Approach

Dissolve boundaries of independent cities that BEA does not disclose values for with their adjacent county.

In [ ]:

csv = pandas.read_csv('51.csv')

In [ ]:
csv.head()

In [ ]:
csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]

In [ ]:
gdf = geopandas.read_file('tl_2019_us_county.shp')

gdf = geopandas.read_file("zip:tl_2019_us_county.zip!tl_2019_us_county.shp")
zipfile = "zip:tl_2019_us_county.zip!tl_2019_us_county.shp"

In [ ]:
virginia_gdf = gdf[gdf.STATEFP=="51"]

In [ ]:
virginia_gdf.plot()

In [ ]:
virginia_gdf.head()

In [ ]:
data = csv

In [ ]:
data.head()

In [ ]:
data['GEOID'] = data.GeoFIPS

In [ ]:
data.GEOID

## gdf records missing income values

In [ ]:
#data = data[data.LineCode==3]
merged_gdf = virginia_gdf.merge(data, on='GEOID')

In [ ]:
merged_gdf.plot()

In [ ]:
merged_gdf.head()

In [ ]:
merged_gdf.shape[0]/3

In [ ]:
matched_names = set(merged_gdf.NAMELSAD)

In [ ]:
matched_names

In [ ]:
missing_gdf = virginia_gdf[~virginia_gdf.GEOID.isin(merged_gdf.GEOID)]

In [ ]:
missing_names = set(missing_gdf.NAMELSAD)

In [ ]:
missing_names

In [ ]:
missing_gdf.plot()

In [ ]:
missing_gdf.shape

In [ ]:
missing_gdf

In [ ]:
#names = missing_gdf.NAME
names = missing_gdf.NAMELSAD

In [ ]:
virginia_income = data[data.GeoName.str.contains(", VA")]

In [ ]:
virginia_income.shape

In [ ]:
virginia_income.head()

In [ ]:
geonames = pandas.unique(virginia_income.GeoName)

In [ ]:
geonames

In [ ]:
class Combination:
    def __init__(self, label):
        n_commas = label.count(",")
        self.label = label
        self.n_commas = n_commas
        if "Fairfax City" in label:
            label = label.replace('Fairfax City', 'Fairfax')
        if n_commas == 1:
            if "Bedford" in label:
                self.county = 'Bedford'
                self.cities = []
            else:
                words = label.split("+")
                county = words[0].strip()
                self.county = county
                self.cities = [words[-1].split(",")[0].strip()+" city"]
        elif n_commas == 2:
            words = label.split(",")
            self.county = words[0].strip()
            cities = words[1]
            cities = cities.split("+")
            
            self.cities = [city.strip()+" city" for city in cities]
        else:
            print('Bad label: ', label)
        self.county = self.county + " County"
        self.NAMELSAD = self.label
        

In [ ]:
combinations_0 = [name for name in geonames if "VA*" in name]
combinations = [Combination(c) for c in combinations_0]

In [ ]:
c0 = combinations[0]
c0.label

In [ ]:
matches = {}
for name in names:
    hits = []
    matches[name] = []
    if "city" in name:
        hits = [comb for comb in combinations if name in comb.cities]
    else:
        hits = [comb for comb in combinations if name == comb.county]
    if hits:
        matches[name] = hits[0].label
    
        
        

## Match combinations to rows of df

In [ ]:
for combination in combinations:
    print("label: ",combination.label)
    comb_names = [combination.county]+combination.cities
    print(comb_names)

In [ ]:
merged_dfs = []
geoids = []
for combination in combinations:
    places = combination.cities + [combination.county]
    print(places)
    rows = virginia_gdf[virginia_gdf.NAMELSAD.isin(places)]
    
    if len(rows) != len(places):
        print('missed:', places)
    #d_df = rows.dissolve(by='STATEFP')
    rd = rows.dissolve(by='STATEFP').head()
    rd['NAMELSAD']= combination.label
    geoid = virginia_income[virginia_income.GeoName==combination.label].GEOID.values[0]
    rd['GEOID'] = geoid
    merged_dfs.append(rd)
    

In [ ]:
rows.head()

In [ ]:
rd.head()

In [ ]:
dissolved = pandas.concat(merged_dfs)

In [ ]:
dissolved.head()

In [ ]:
d_merge = dissolved.merge(virginia_income, on='GEOID')

In [ ]:
d_merge.head()

In [ ]:
d_merge.shape

In [ ]:
d_merge.plot()

In [ ]:
merged_gdf.plot()

In [ ]:
final_gdf = pandas.concat([merged_gdf, d_merge])
final_gdf.plot()

In [ ]:
final_gdf.shape

In [ ]:
virginia_gdf.shape

In [ ]:
final_gdf.head()

In [ ]:
final_gdf.tail()

In [ ]:
final_gdf.plot(column='1978')

In [ ]:
final_gdf['1979'] = final_gdf['1979'].astype(int)

In [ ]:
final_gdf.plot(column='1979', legend=False, scheme='Quantiles', k=9)

In [ ]:
final_gdf.columns

In [ ]:
final_gdf.head()

In [ ]:
final_gdf.shape

In [ ]:
327859/37657 * 1000

In [ ]:
318/3


In [ ]:
final_gdf.STATEFP='51'

In [ ]:
final_gdf[final_gdf.NAME=='Charlottesville']

In [ ]:
727271*1000/93879

## Reverse Matching

Which income records do not match a record in the shapefile

In [ ]:
virginia_gdf = gdf[gdf.STATEFP=="51"]

In [ ]:
#data = data[data.LineCode==3]
merged_df = data.merge(virginia_gdf, on='GEOID')

In [ ]:
merged_df.shape

In [ ]:
missing_df = data[~data.GEOID.isin(merged_df.GEOID)]
#~virginia_gdf.GEOID.isin(merged_gdf.GEOID)

In [ ]:
set(missing_df.GeoName)

In [ ]:
pcincome_gdf = geopandas.read_file('pcincome0.shp')

In [ ]:
pcincome_gdf.head()

In [ ]:
final_gdf.head()

In [ ]:
final_gdf.columns

In [ ]:
pcincome_gdf.columns

In [ ]:
common = set(pcincome_gdf.columns).intersection(final_gdf.columns)

In [ ]:
common

In [ ]:
us = pcincome_gdf[common]

In [ ]:
va = final_gdf[common]


In [ ]:
va.reset_index(inplace=True)

In [ ]:
dup = va[va[["NAME", "GEOID", "1969"]].duplicated()]

In [ ]:
dup.NAME

In [ ]:
dup

In [ ]:
va = va.drop(dup.index)

In [ ]:
usva = pandas.concat([us,va])

In [ ]:
usva.plot()

In [ ]:
usva.shape

In [ ]:
9027/3

In [ ]:
years = [str(year) for year in range(1969,2018)]
dtypes = list(zip(years,[usva[year].dtype for year in years]))
dtypes

In [ ]:
usva.to_file('usva.shp')

In [ ]:
usva.head()

In [ ]:
usva.columns

In [ ]:
namedf = usva[['GeoName', 'STATEFP', '1969']]

In [ ]:
namedf[namedf.duplicated()]

In [ ]:
namedf[namedf.GeoName=='Bedford, VA*']

## Wisconsin

In [ ]:

csv = pandas.read_csv('55.csv')

In [ ]:
csv.head()

In [ ]:
csv[csv.GeoName.str.startswith('Shawano')]

In [ ]:
csv[csv.GeoName.str.startswith('Menominee')]

In [ ]:
csv = csv.replace('(NA)', 0)

In [ ]:
menominee = csv.iloc[[120, 121, 122]]
shawano = csv.iloc[[177, 178, 179]]
combined = csv.iloc[[219, 220, 221]]

In [ ]:
years = [str(y) for y in range(1989, 2018)]

In [ ]:
menominee[years] = menominee[years].astype(int)
shawano[years] = shawano[years].astype(int)
combined[years] = combined[years].astype(int)

In [ ]:
cindex = [ combined.columns.values.tolist().index(y) for y in years]

for c,y in zip(cindex,years):
    #print(menominee[y].dtype)
    #print(shawano[y].dtype)
    #print(combined[y].dtype)
    combined.iloc[0,c] = menominee.iloc[0,c] + shawano.iloc[0,c]
    combined.iloc[1,c] = menominee.iloc[1,c] + shawano.iloc[1,c]
    combined.iloc[2,c] = int(combined.iloc[0,c]*1000/ combined.iloc[1,c])
    csv.iloc[219,c] = combined.iloc[0,c]
    csv.iloc[220,c] = combined.iloc[1,c]
    csv.iloc[221,c] = combined.iloc[2,c]




In [ ]:
combined

In [ ]:
csv.iloc[[219,220,221]]

## Drop shawano and menomiee from csv

In [ ]:
drop_ids = [120, 121, 122, 177, 178, 179]
csv = csv.drop(drop_ids)

In [ ]:
years = [str(y) for y in range(1969, 2018)]
csv[years] = csv[years].astype(int)

In [ ]:
for year in range(1969,2018):
    print(csv[str(year)].dtype)
    

## Dissolve shawano and menominee geometries in gdf

dissolve
and assign geofips

In [ ]:
gdf = geopandas.read_file("tl_2019_us_county.shp")

In [ ]:
wisconsin_gdf = gdf[gdf.STATEFP=="55"]

In [ ]:
wisconsin_gdf.plot()

In [ ]:
wisconsin_gdf[wisconsin_gdf.NAME=='Shawano']

In [ ]:
wisconsin_gdf[wisconsin_gdf.NAME=='Menominee']

In [ ]:
combined = wisconsin_gdf[wisconsin_gdf.COUNTYFP.isin(['115', '078'])].dissolve(by='STATEFP')

In [ ]:
combined.plot()

In [ ]:
combined.head()

In [ ]:
combined['NAME'] = "Shawano+Menominee"
combined['NAMELSAD'] = "Shawano+Menominee Counties"

In [ ]:
combined.head()

In [ ]:
combined['GEOID'] = '55901'

In [ ]:
combined.head()

In [ ]:
wisc0 = wisconsin_gdf[~wisconsin_gdf.COUNTYFP.isin(['115', '078'])]

In [ ]:
wisc0.plot()

In [ ]:
wisc1 = pandas.concat([wisc0, combined])

In [ ]:
wisc1.plot()

In [ ]:
wisc1.head()

In [ ]:
wisc1.shape

In [ ]:
wisconsin_gdf.shape

## Merge gdf and csv

In [ ]:
csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]


In [ ]:
wisc1.head()

In [ ]:
wisc1.STATEFP='55'

In [ ]:
data = csv

In [ ]:
data.head()

In [ ]:
data['GEOID'] = data.GeoFIPS

In [ ]:
data.GEOID

## gdf records missing income values

In [ ]:
#data = data[data.LineCode==3]
merged_gdf = wisc1.merge(data, on='GEOID')

In [ ]:
set(wisc1.GEOID)

In [ ]:
merged_gdf.plot()

In [ ]:
merged_gdf.head()

In [ ]:
merged_gdf.shape[0]/3

In [ ]:
matched_names = set(merged_gdf.NAMELSAD)

In [ ]:
matched_names

In [ ]:
missing_gdf = wisc1[~wisc1.GEOID.isin(merged_gdf.GEOID)]

In [ ]:
missing_names = set(missing_gdf.NAMELSAD)

In [ ]:
missing_names

In [ ]:
missing_gdf.head()

In [ ]:
usva = geopandas.read_file('usva.shp')

In [ ]:
us_no_wisc = usva[usva.STATEFP!='55']

In [ ]:
us = pandas.concat([merged_gdf, us_no_wisc])

In [ ]:
us.shape

In [ ]:
9098/3

In [ ]:
us_no_wisc.plot()

In [ ]:
us.plot()

In [ ]:
us.shape

In [ ]:
9240/3

In [ ]:
us.head()

In [ ]:
for y in range(1969, 2018):
    print(us[str(y)].dtype)

In [ ]:
us.to_file('usincome.shp')

# Other

## Issue from BEA Income Data:
<LI>*&nbsp;Cibola, NM was separated from Valencia in June 1981, but in these estimates, Valencia includes Cibola through the end of 1981.</LI>
<LI>*&nbsp;La Paz County, AZ was separated from Yuma County on January 1, 1983. The Yuma, AZ MSA contains the area that became La Paz County, AZ through 1982 and excludes it beginning with 1983.</LI>

<LI>*&nbsp;Broomfield County, CO, was created from parts of Adams, Boulder, Jefferson, and Weld counties effective November 15, 2001. Estimates for Broomfield county begin with 2002.</LI>


### Approach

- combine Cibola NM with Valencia for 1981-2017
- combine La Paz County AZ with Yuma for 1983-2017
- combine Broomfield County with Boulder CO all years (<2002 Boulder, >2002 Boulder+Broomfield)



In [ ]:
import pandas
import geopandas
%matplotlib inline

## NM

In [ ]:
csv = pandas.read_csv('35.csv')

In [ ]:
csv = csv.replace('(NA)', 0)

In [ ]:
cibola = csv[csv.GeoName.str.startswith('Cibola')]
valencia = csv[csv.GeoName.str.startswith('Valencia')]

In [ ]:
cibola

In [ ]:
valencia

In [ ]:
years = [ str(y) for y in range(1969, 2018)]

In [ ]:
combined = cibola.copy()
cibola[years] = cibola[years].astype(int)
valencia[years] = valencia[years].astype(int)
combined[years] = combined[years].astype(int)

In [ ]:
cindex = [ combined.columns.values.tolist().index(y) for y in years]

for c,y in zip(cindex,years):
    #print(menominee[y].dtype)
    #print(shawano[y].dtype)
    #print(combined[y].dtype)
    
    
    combined.iloc[0,c] = valencia.iloc[0,c] + cibola.iloc[0,c]
    combined.iloc[1,c] = valencia.iloc[1,c] + cibola.iloc[1,c]
    combined.iloc[2,c] = int(combined.iloc[0,c]*1000/ combined.iloc[1,c])
    #csv.iloc[219,c] = combined.iloc[0,c]
    #csv.iloc[220,c] = combined.iloc[1,c]
    #csv.iloc[221,c] = combined.iloc[2,c]


In [ ]:
combined

## Drop Cibola and Valencia and Update Combined Fields

In [ ]:
csv = csv.drop([12, 13, 14, 99, 100, 101])
combined['GeoName'] = 'Cibola+Valencia'
combined['GeoFIPS'] = '"35061"'

In [ ]:
combined = combined[csv.columns]

csv = pandas.concat([csv, combined])

In [ ]:
csv.head()

In [ ]:
csv.tail()

In [ ]:
gdf = geopandas.read_file("tl_2019_us_county.shp")

In [ ]:
nm_gdf = gdf[gdf.STATEFP=="35"]

In [ ]:
nm_gdf.plot()

In [ ]:
nm_gdf[nm_gdf.NAME.isin(['Valencia', 'Cibola'])]

In [ ]:
combined = nm_gdf[nm_gdf.NAME.isin(['Valencia', 'Cibola'])].dissolve(by='STATEFP')

In [ ]:
combined.plot()

In [ ]:
combined['NAME'] = 'Cibola+Valencia'
combined['NAMELSAD'] = 'Cibola+Valencia Counties'
combined['STATEFP'] = 35

In [ ]:
combined['GEOID'] =  '35061' # Valencia

In [ ]:
nm0 = nm_gdf[~nm_gdf.COUNTYFP.isin(['006', '061'])]

In [ ]:
nm0.plot()

In [ ]:
nm1 = pandas.concat([nm0, combined])

In [ ]:
nm1.plot()

In [ ]:
nm_gdf.shape

In [ ]:
nm1.shape

In [ ]:
csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]


In [ ]:
data = csv
data['GEOID'] = data.GeoFIPS

In [ ]:
merged_gdf = nm1.merge(data, on='GEOID')

In [ ]:
merged_gdf.shape

In [ ]:
merged_gdf.head()

In [ ]:
us = geopandas.read_file('usincome.shp')

In [ ]:
us1 = pandas.concat([us[us.STATEFP!='35'], merged_gdf])

In [ ]:
us1.shape

In [ ]:
us1.head()

In [ ]:
us1.tail()

In [ ]:
9237/3

In [ ]:
merged_gdf.head()

In [ ]:
us1[us1.NAME.str.startswith('Cibo')]

##  Arizona

- combine La Paz County AZ with Yuma for 1983-2017


In [ ]:
csv = pandas.read_csv('04.csv')


In [ ]:
csv.head()

In [ ]:
csv = csv.replace('(NA)', 0)

In [ ]:
csv[csv.GeoName.str.startswith('La Paz')]

In [ ]:
csv[csv.GeoName.str.startswith('Yuma')]

In [ ]:
la_paz = csv.iloc[[21, 22, 23]]
yuma = csv.iloc[[45, 46, 47]]
combined = yuma.copy()
years = [str(y) for y in range(1969, 2018)]
la_paz[years] = la_paz[years].astype(int)
yuma[years] = yuma[years].astype(int)
combined[years] = combined[years].astype(int)

In [ ]:
cindex = [ combined.columns.values.tolist().index(y) for y in years]

for c,y in zip(cindex,years):
    #print(menominee[y].dtype)
    #print(shawano[y].dtype)
    #print(combined[y].dtype)
    combined.iloc[0,c] = la_paz.iloc[0,c] + yuma.iloc[0,c]
    combined.iloc[1,c] = la_paz.iloc[1,c] + yuma.iloc[1,c]
    combined.iloc[2,c] = int(combined.iloc[0,c]*1000/ combined.iloc[1,c])

In [ ]:
combined

In [ ]:
csv = csv.drop([21, 22, 23, 45, 46, 47])
combined['GeoName'] = 'Yuma+La Paz'
combined['GeoFIPS'] = '"04027"'
combined['STATEFP'] = 4

In [ ]:
combined = combined[csv.columns]

In [ ]:
combined.head()

In [ ]:
csv = pandas.concat([csv, combined])

In [ ]:
csv.shape

In [ ]:
csv.head()

In [ ]:
gdf = geopandas.read_file("tl_2019_us_county.shp")
az_gdf = gdf[gdf.STATEFP=='04']

az_gdf.plot()

In [ ]:
az_gdf[az_gdf.NAME=='La Paz']

In [ ]:
az_gdf[az_gdf.NAME=='Yuma']

In [ ]:
combined = az_gdf[az_gdf.COUNTYFP.isin(['027', '012'])].dissolve(by='STATEFP')

In [ ]:
combined.plot()

In [ ]:
combined['NAME']= "Yuma+La Paz"
combined['NAMELSAD'] = "Yuma+La Paz Counties"


In [ ]:
combined.head()

In [ ]:
az0 = az_gdf[~az_gdf.COUNTYFP.isin(['027', '012'])]
az0.plot()

In [ ]:
az1 = pandas.concat([az0, combined])

In [ ]:
az1.plot()

## Merge gdf and csv

In [ ]:
csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]


In [ ]:
data = csv
data.head()

In [ ]:
data['GEOID'] = data.GeoFIPS

In [ ]:
merged_gdf = az1.merge(data, on='GEOID')
merged_gdf['STATEFP'] = '04'

In [ ]:
merged_gdf.plot()

In [ ]:
us2 = pandas.concat([us1[us1.STATEFP!='04'], merged_gdf])

In [ ]:
us2[us2.STATEFP=='04'].shape

In [ ]:
us2.shape[0]/3

## Colorado

- combine Broomfield County with Boulder CO all years (<2002 Boulder, >2002 Boulder+Broomfield)


In [ ]:
csv = pandas.read_csv('08.csv')

In [ ]:
csv.head()

In [ ]:
csv[csv.GeoName.str.startswith('Boulder')]

In [ ]:
csv[csv.GeoName.str.startswith('Broom')]

In [ ]:
csv = csv.replace('(NA)', 0)
boulder = csv.iloc[[21, 22, 23]]
broomfield = csv.iloc[[24, 25, 26]]
combined = boulder.copy()
years = [str(y) for y in range(1989, 2018)]
boulder[years] = boulder[years].astype(int)
broomfield[years] = broomfield[years].astype(int)
combined[years] = combined[years].astype(int)

In [ ]:
cindex = [ combined.columns.values.tolist().index(y) for y in years]

for c,y in zip(cindex,years):
    #print(menominee[y].dtype)
    #print(shawano[y].dtype)
    #print(combined[y].dtype)
    combined.iloc[0,c] = boulder.iloc[0,c] + broomfield.iloc[0,c]
    combined.iloc[1,c] = boulder.iloc[1,c] + broomfield.iloc[1,c]
    combined.iloc[2,c] = int(combined.iloc[0,c]*1000/ combined.iloc[1,c])
    

In [ ]:
csv = csv.drop([21, 22, 23, 24, 25, 26])
years = [str(y) for y in range(1969, 2018)]
csv[years] = csv[years].astype(int)

In [ ]:
combined['GeoName'] = 'Boulder+Broomfield'
combined['GeoFIPS'] = '"08013"'

In [ ]:
combined = combined[csv.columns]

csv = pandas.concat([csv, combined])

In [ ]:
csv.head()

In [ ]:
csv.tail()

In [ ]:
csv[csv.GeoName.str.startswith('Bould')]

In [ ]:
gdf = geopandas.read_file("tl_2019_us_county.shp")

co_gdf = gdf[gdf.STATEFP=='08']

co_gdf.plot()

In [ ]:
co_gdf[co_gdf.NAME=='Boulder']

In [ ]:
co_gdf[co_gdf.NAME=='Broomfield']

In [ ]:
combined = co_gdf[co_gdf.COUNTYFP.isin(['013', '014'])].dissolve(by='STATEFP')

In [ ]:
combined.plot()

In [ ]:
combined.head()

In [ ]:
combined['NAME'] = "Boulder+Bloomfield"
combined['NAMELSAD'] = "Boulder+Bloomfield Counties"


In [ ]:
combined['GEOID'] = '08013'

In [ ]:
combined.columns

In [ ]:
co0 = co_gdf[~co_gdf.COUNTYFP.isin(['013', '014'])]
co0.plot()

In [ ]:
co1 = pandas.concat([co0, combined])
co1.plot()

In [ ]:
co_gdf.plot()

In [ ]:
co1.shape

In [ ]:
co_gdf.shape

In [ ]:
csv['GeoFIPS'] = [fip.strip().replace("\"", "") for fip in csv.GeoFIPS]


In [ ]:
data = csv
data['GEOID'] = data.GeoFIPS

In [ ]:
merged = co1.merge(data, on='GEOID')
merged['STATEFP'] = '08'

In [ ]:
merged[merged.NAME.str.startswith('Bould')]['STATEFP']

In [ ]:
co1.shape

In [ ]:
merged.shape

In [ ]:
data.shape

In [ ]:
merged.plot()

In [ ]:
merged.head()

In [ ]:
us2.head()

In [ ]:
us3 = pandas.concat([us2[us2.STATEFP!='08'], merged])

In [ ]:
us3.shape[0]/3

In [ ]:
us2.shape[0]/3

In [ ]:
us3

In [ ]:
us3[us3.STATEFP=='08'].tail()

In [ ]:
us3[us3.NAME.str.startswith('Boulder')]

In [ ]:
us3.columns

In [ ]:
columns = us3.columns
print(columns)


In [ ]:
us3.shape

In [ ]:
columns = us3.columns.values.tolist()
columns

In [ ]:
us3.loc[:,'Unnamed: 0']

In [ ]:
us3.shape

In [ ]:
us4 = us3.drop(columns=['Unnamed: 0', 'Unnamed_ 0', 'IndustryCl'])

In [ ]:
us4.shape

In [ ]:
for year in range(1969, 2018):
    year = str(year)
    us4[year] = us4[year].astype('int')

In [ ]:
us4.to_file('usincome_final.shp')

In [ ]:
us4.shape

## Shrinking file size
We are duplicating the shapes three times (once for each attribute)

Split out the attributes from the geometries, two different dataframes

In [ ]:
gdf = gpd.read_file('usincome_final.shp')

In [ ]:
uscountyincome = pandas.DataFrame(gdf.drop(columns='geometry'))

In [ ]:
uscountyincome.to_csv('uscountyincome.csv')

In [ ]:
gdf.head()

In [ ]:
gdf.columns

In [ ]:
gdf.LineCode

In [ ]:
gdf = gdf[gdf.LineCode==3]

In [ ]:
gdf.shape

In [ ]:
%pylab inline

In [ ]:
gdf.plot(column='2017')

In [ ]:
gdf.to_file("uscountypcincome.gpkg", layer='pcincome', driver="GPKG")

In [ ]:
gdf.columns

In [ ]:
gdf.shape